<a href="https://colab.research.google.com/github/caroheymes/jedha_final_project/blob/master/features_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Packages

In [ ]:
!pip install spacy -q
!python -m spacy download fr_core_news_sm -q
!pip install unidecode
!pip install unidecode
!pip install wordcloud -q
!pip install stylecloud
!pip install matplotlib==3.4
!pip install random2


In [2]:

import os
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

import fr_core_news_sm
import string
import unidecode
from unidecode import unidecode
import re
import pandas as pd
import json
from fnmatch import fnmatch
import random

# Import matplotlib and wordcloud 
import chardet
import spacy
import fr_core_news_sm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import wordcloud
from spacy.lang.fr.stop_words import STOP_WORDS
import numpy as np
from PIL import Image
from collections import Counter


import warnings; warnings.filterwarnings(action='ignore')
import time


import seaborn as sns
%matplotlib inline
large = 18; med = 14; small = 11
params = {'axes.titlesize': large,
          'legend.fontsize': small,
          'figure.figsize': (16, 8),
          'axes.labelsize': small,
          'axes.titlesize': small,
          'xtick.labelsize': small,
          'ytick.labelsize': small,
          'figure.titlesize': med}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

Mounted at /content/drive/


# Load keyword files

All keywords have been exported from Google Ads and already cleaned

In [3]:
#Loading kewywords
kw = pd.read_csv('/content/drive/MyDrive/111_goldfinger/data_collection/results/correct_keywords.csv')
kw['kw_len'] = [len(x.split(' ')) for x in kw.keyword]
kw = kw.sort_values(by='kw_len', ascending=False)[['keyword', 'volume']].values
kw[1:5]

array([['recherche emploi auxiliaire de vie en maison de retraite',
        500.0],
       ['technicienne en service de garde en milieu scolaire emploi',
        50.0],
       ['conducteur accompagnateur de personnes à mobilité réduite pôle emploi',
        50.0],
       ['recrutement executif chef de cuisine hôtellerie de luxe', 500.0]],
      dtype=object)

# Load positions and urls from 0 to 100

In [5]:
from fnmatch import fnmatch
#list files related to serp data ins "liste"
root = '/content/drive/MyDrive/111_goldfinger/data_collection'
pattern = "*serp*"
i=0
liste = []
for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            liste.append(name)
            i+=1
liste.append('last.csv')

In [6]:
liste

['serp.csv',
 'serp1.csv',
 'serp2.csv',
 'serp3.csv',
 '20_serp1.csv',
 '20_serp3.csv',
 '20_serp2.csv',
 'last.csv']

In [7]:
os.chdir('/content/drive/MyDrive/111_goldfinger/data_collection')
df = pd.DataFrame()
for e in liste:
  df = pd.concat([df, pd.read_csv(e)], axis=0)

In [ ]:
print(df.shape)

(116675, 1541)


In [8]:
df.sample()

,Unnamed: 0,gl,start,searchTerms,rank,title,snippet,displayLink,link,queryTime,totalResults,cacheId,count,cseName,cx,formattedSearchTime,formattedTotalResults,formattedUrl,htmlFormattedUrl,htmlSnippet,htmlTitle,inputEncoding,kind,outputEncoding,pagemap,safe,searchTime,startIndex,cse_thumbnail,metatags,cse_image,listitem,monetaryamount,quantitativevalue,jobposting,propertyvalue,organization,postaladdress,sitenavigationelement,og:image,...,documentcategory,institution,parldatarssuri,parldatauri,documenttitle,documentid,build,governmentorganization,contactpoint,dd-trace-time,dd-trace-id,ids_page-contact_cap-emploi,og:titre,wpsso-begin,dcterms:accessrights,dcterms:service,Post,crossmarkdomains_1_,crossmarkdomains_2_,crossmarkmajorversiondate,crossmarkdomainexclusive,elsevierwebpdfspecifications,doi,<modeltra>,created with,<modelcod>,<extension>,lastedited with,<model>,pe number,<modelvie>,<type>,footerpath,<fdr>,business:contact_data:postal_code,business:contact_data:locality,business:contact_data:region,product:price:currency,product:price:amount,business:contact_data:street_address
906,7,fr,21.0,Emploi PREPARATEUR DE COMMANDES H/F - Vourles,28.0,Offres d'emploi Mornant (69440) - Alentoor,Toutes les offres d'emploi à Mornant et aux en...,www.alentoor.fr,https://www.alentoor.fr/mornant/emploi,2021-11-25 13:49:53.923127+00:00,5300.0,fI0kfDX7YSYJ,10.0,chiotte,e7653c0516e574854,0.3,"5,300",https://www.alentoor.fr/mornant/emploi,https://www.alentoor.fr/mornant/<b>emploi</b>,Toutes les offres d&#39;<b>emploi</b> à Mornan...,Offres d&#39;<b>emploi</b> Mornant (69440) - A...,utf8,customsearch#result,utf8,{'cse_thumbnail': [{'src': 'https://encrypted-...,off,0.30417,21.0,[{'src': 'https://encrypted-tbn0.gstatic.com/i...,"[{'msapplication-tilecolor': '#9addf7', 'og:im...",[{'src': 'https://www.alentoor.fr/assets/img/s...,"[{'item': 'Emploi Rhône-Alpes', 'name': 'Emplo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.alentoor.fr/assets/img/social/face...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# There are more than 1500 columns such as 'fileFormat',
# 'formattedSearchTime', 'formattedTotalResults', 'formattedUrl',
# 'htmlFormattedUrl', 'htmlSnippet', 'htmlTitle', 'inputEncoding', 'kind',
# 'mime', 'outputEncoding', 'pagemap', 'safe', 'searchTime', 'startIndex',
# 'cse_thumbnail', 'metatags', 'cse_image', 'listitem', 'BreadcrumbList',
# 'article', 'msapplication-tilecolor', 'og:image', 'og:type',
# 'og:image:width', 'fb:app_id', 'theme-color', 'viewport', 'og:title',
# 'og:image:height', 'og:url', 'og:description', 'format-detection',
# 'twitter:app:id:googleplay'... We'll then select the most useful for ou analysis

df = df[['searchTerms', 'rank', 'title','displayLink', 'link']] 

In [10]:
# Drop records with no availaible url
df.dropna(subset=['link'], inplace=True)
print(df.shape)

(108776, 5)


# Load pages from 0 to 100

In [11]:
# Loading all scraped pages from position 0 to 100

pages_liste = ['out_of_top.csv', 'lot1.csv', 'lot2.csv', 'lot3.csv', 'lot4.csv', '11_to_20.csv'] #préférer les fichiers json

In [ ]:
os.chdir('/content/drive/MyDrive/111_goldfinger/data_collection/all_pages')
dfr = pd.DataFrame()
for e in pages_liste:
  print(e)
  dfr = pd.concat([dfr, pd.read_csv(e,error_bad_lines=False, skip_blank_lines=True, engine="python")], axis=0)


In [13]:
dfr.shape

(94120, 12)

In [14]:
# Merge position and pages content
data = dfr.merge(df, left_on='raw_url', right_on='link', how='left')  #à reprendre quand integ en json
data.sample()


,canonical,content,h1,h2,h3,h4,header_links_href,header_links_text,meta_desc,nav_links_text,raw_url,title_x,searchTerms,rank,title_y,displayLink,link
77772,https://www.offresdinterim.com/,Offresdinterim.com est un site d'offres d'empl...,Accueil,"Notre mission,@@c'est la votre@@Nos Annonces",Trouvez la mission que vous recherchez !@@Vous...,NaN,https://www.offresdinterim.com/mon-compte/@@ht...,Se connecter@@S'inscrire@@Mon compte@@Espace A...,NaN,Trouver un job@@Tarification@@A propos@@Contac...,https://www.offresdinterim.com/,"Accueil - Offres d'Interim, CDD, CDI en France",Emploi CHAUFFEUR SPL FRIGO H/F - Saint-Aubin-s...,9.0,"Offres d'Interim, CDD, CDI en France: Accueil",www.offresdinterim.com,https://www.offresdinterim.com/


In [ ]:
data.sample()

,canonical,content,h1,h2,h3,h4,header_links_href,header_links_text,meta_desc,nav_links_text,raw_url,title_x,searchTerms,rank,title_y,displayLink,link
204576,NaN,x x Etes-vous sur de vouloir supprimer votre c...,Offres d'emploi dans la region Ile-de-France,Main navigation@@Les offres d'emploi :,Departements de la region Ile-de-France :@@Aut...,Menu utilisateur@@Mot de passe oublie@@Candida...,/@@#show-block-start-main-menu@@#hide-block-st...,Afficher -- Main navigation@@Masquer -- Main n...,NaN,Afficher -- Main navigation@@Masquer -- Main n...,https://www.startpeople.fr/offres-emploi/ile-d...,Offres d'emploi dans la region Ile-de-France |...,Emploi FACTEUR A VELO H/F - Palaiseau,7.0,Offres d'emploi dans la région Île-de-France |...,www.startpeople.fr,https://www.startpeople.fr/offres-emploi/ile-d...


In [15]:
data = data.drop(columns = ['title_y']).rename(columns={'title_x' :'title'})
data.drop_duplicates(inplace=True)

In [16]:
data.shape

(131678, 16)

In [18]:
os.chdir('/content/drive/MyDrive/111_goldfinger/data_processing')
data.to_csv('brut_data.csv')

# Semantic scores calculation 

In [10]:
data = pd.read_csv('/content/drive/MyDrive/111_goldfinger/data_processing/brut_data.csv')
data = data[~data['rank'].isna()] #drop lines with no position

In [12]:
data.drop_duplicates(inplace=True)

In [14]:
#content
data['content_len']               = [len(str(x)) for x in data.content]

#title
data['title_len_char']             = [len(str(x)) for x in data.title]

#h1
data['h1_len']                    = [len(str(x).split(' ')) for x in data.h1]

 
#h2, h3
data['nb_h2'] 					          = [len(re.findall('@@',str(x))) for x in data.h2]
data['nb_h3'] 					          = [len(re.findall('@@',str(x))) for x in data.h3]

#links
data['nb_links']                  = [len(re.findall('@@',str(x))) for x in data.nav_links_text]
data['has_canonical']             = [1 if isinstance(x, str) else 0 for x in data.canonical]
data['is_top_ten']                = [1 if x < 11 else 0 for x in data['rank']]


# data.to_csv('+++basic.csv')       

In [17]:
data = data.drop(columns= ['Unnamed: 0', 'canonical', 'h2', 'h3', 'h4', 'meta_desc',
        'title', 'searchTerms', 'nav_links_text', 'rank'])

In [18]:
data.to_csv('data_without_score.csv')

adding semantic scores for title & content

In [21]:
# Check if the kw list is loaded
kw[random.randint(0, len(kw))]

array(['chauffeur livreur véhicule léger', 500.0], dtype=object)

In [ ]:
# Custom function to estimate semantic relevance
# Goal :  evaluate if any of keywords is in document 
# Returns a score based on number of (Google Ads) keywords in text x volume (of search) / text lenght

def word_score(doc, kw):
  score = 0
  tokens = []
  try:
    if isinstance(doc, str):
      doc = doc.lower()
      if '@@' in doc:
        doc = doc.split('@@')
        
      initial_len = len(doc.split(' '))
      
      for e in kw:
        if doc is not None:
          try:
            if len(doc.replace(e[0], '')) < len(doc):
              score += e[1]
              tokens += e[0] + '@@'
              doc = doc.replace(e[0], '')
          except:
            pass

      score = score / initial_len
  except:
    score = 0
    tokens = []

  
  return score


In [ ]:
data['content_score']             = [word_score(x, kw) for x in data.content]
data['title_score']               = [word_score(x, kw) for x in data.title]


In [52]:
data.sample()

,Unnamed: 0,content,h1,header_links_href,header_links_text,raw_url,displayLink,link,content_len,title_len_char,h1_len,nb_h2,nb_h3,nb_links,has_canonical,is_top_ten,content_score,title,title_score
89060,89975,Trouver Employeur Entrer Filtres Date de publi...,Emploi Stage maritime Marseille,/@@/employer/jobposting@@/auth/login/phone?bac...,NaN,https://fr.jooble.org/emploi-stage-maritime/Ma...,fr.jooble.org,https://fr.jooble.org/emploi-stage-maritime/Ma...,5594,95,4,0,0,19,1,1,56.561086,Besoin d'urgence: Stage maritime a Marseille o...,0.0


In [45]:
data.to_csv('/content/drive/MyDrive/111_goldfinger/data_processing/data_w_score.csv')

# Append page rank data estimates

In [81]:
data = pd.read_csv('/content/drive/MyDrive/111_goldfinger/data_processing/data_w_score.csv')

In [82]:
pr = pd.read_csv('/content/drive/MyDrive/111_goldfinger/data_collection/results/page_rank_done.csv').drop(columns='Unnamed: 0')

In [83]:
data = data.merge(pr, left_on='displayLink', right_on='dns', how='left')

In [84]:
data = data.drop(columns=['Unnamed: 0', 'content', 'h1', 'header_links_href', 'header_links_text',
       'raw_url', 'displayLink', 'link', 
        'title',  'dns']).rename(columns={'score' : 'page_rank'}).drop_duplicates()

In [88]:
data.drop_duplicates().fillna(0).drop(columns=['Unnamed: 0.1']).to_csv('/content/drive/MyDrive/111_goldfinger/data_processing/experimental.csv')

# Fetch  & append Majestic Rank, Referring Subnets, Referring IPs data

Builtwith offers a splendid api for fetching tech data points.
We're going to collect some additionnal features (and check if they are relevant for analysis):
```
* technologies'  = len(r.json()['Results'][0]['Result']['Paths'][0]['Technologies'])
* 'mjrank'       = r.json()['Results'][0]['Meta']['MJRank']                           #Majestic Rank between 1 and 1,000,000
* 'a_rank'        = r.json()['Results'][0]['Meta']['ARank']                           #Page Rank between 1 and 100,000,000
* 'ref_sn'        = r.json()['Results'][0]['Attributes']['RefSN']                     #Referring Subnets to Domain from Majestic
* 'ref_ip'        = r.json()['Results'][0]['Attributes']['RefIP']                     #Referring IPs to Domain from Majestic
```



Selecting most frequent domains (> 93% of dataset)

In [92]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/111_goldfinger/data_processing/data_w_score.csv')

In [26]:
#Enumeration of unique domains
df = df[['displayLink', 'link']]
df.displayLink.unique()
print(len(df.displayLink.unique()), 'unique domains')

4353 unique domains


In [33]:
# What are the dns that come up most often?
count_dns = df.displayLink.value_counts().to_frame()
count_dns['cum_sum'] = count_dns['displayLink'].cumsum()
count_dns['cum_perc'] = 100*count_dns['cum_sum']/count_dns['displayLink'].sum()
count_dns = count_dns[count_dns.displayLink	 > 10]
count_dns


,displayLink,cum_sum,cum_perc
www.manpower.fr,15156,15156,11.596643
fr.indeed.com,6944,22100,16.909857
www.meteojob.com,5962,28062,21.471693
candidat.pole-emploi.fr,5584,33646,25.744302
fr.jooble.org,5154,38800,29.687895
...,...,...,...
www.acapars.com,11,121533,92.991208
www.viseointerim.fr,11,121544,92.999625
www.helpemploi.fr,11,121555,93.008042
www.rungis-emplois.com,11,121566,93.016458


In [ ]:
import os
import requests
from time import sleep
bw_key = open("/content/drive/MyDrive/111_goldfinger/data_collection/bw_key", "r").read()
tech_features = pd.DataFrame()

for elem in count_dns.index:
  e = elem.replace('www.', '')
  url = 'https://api.builtwith.com/v19/api.json?KEY='  + bw_key + '&LOOKUP=' +e

  r = requests.get(url)
  try:
    technologies  = len(r.json()['Results'][0]['Result']['Paths'][0]['Technologies'])
  except:
    technologies =0
  try:
    majestic      = r.json()['Results'][0]['Meta']['Majestic'] 
  except:
    majestic = 0
  try:
    mjrank         = r.json()['Results'][0]['Attributes']['MJRank']
  except:
    mjrank = 0
  try:
    a_rank        = r.json()['Results'][0]['Meta']['ARank']
  except:
    a_rank = 0
  try:                              
    ref_sn        = r.json()['Results'][0]['Attributes']['RefSN']
  except:
    ref_sn = 0
  try:                  
    ref_ip        = r.json()['Results'][0]['Attributes']['RefIP']
  except:
    ref_ip = 0
  try:                 
    spend         = r.json()['Results'][0]['Result']['Spend']
  except:
    spend = 0

  tech = {
  'dns'           : elem,
  'technologies'  : technologies,
  'majestic'      : majestic,
  'mjrank'        : mjrank,                          
  'a_rank'        : a_rank,                          
  'ref_sn'        : ref_sn,                    
  'ref_ip'        : ref_ip,                    
  'spend'         : spend,                    
  }

  tech_features = pd.concat([tech_features,pd.DataFrame.from_dict(data=tech, orient='index').T], axis=0 )
  sleep(0.5)
tech_features.to_csv('/content/drive/MyDrive/111_goldfinger/data_collection/builtwith.csv')
tech_features

In [94]:
# Merging DataFrames
df_with_tech= df.merge(tech_features, left_on='displayLink', right_on='dns', how='left')


In [99]:
df_with_tech = df_with_tech.drop(columns= ['Unnamed: 0', 'Unnamed: 0.1', 'content', 'h1', 'header_links_href',
       'header_links_text', 'raw_url', 'displayLink', 'link','title','dns'])


In [100]:
df_with_tech.sample().T

,120820
content_len,2538
title_len_char,108
h1_len,1
nb_h2,6
nb_h3,0
nb_links,21
has_canonical,1
is_top_ten,0
content_score,118.765
title_score,33.3333


In [102]:
df_with_tech.to_csv('tech.csv')